In [1]:
import numpy as np
import os


# Main Parameters
VOXEL_SIZE = 0.05
k_1 = 1
k_2 = 1
threshold = 0.01

In [2]:
from conversion import np_point_cloud2_pcd, pcd2np_point_cloud
from input_output import load_point_cloud

# Load Point Clouds and prepocess data
PC1 = "assets/bun000_Cloud.las"
PC2 = "assets/bun045_Cloud.las"
output_path = "output"

lst_PC = [load_point_cloud(PC1), load_point_cloud(PC2)]
lst_acc = [0.01, 0.01]
reference_pc = np.vstack(lst_PC)

ds_reference = np_point_cloud2_pcd(reference_pc).voxel_down_sample(
    voxel_size=VOXEL_SIZE
)
ds_sub_reference = np_point_cloud2_pcd(reference_pc).voxel_down_sample(
    voxel_size=VOXEL_SIZE / 2
)
ds_reference = pcd2np_point_cloud(ds_reference)
ds_sub_reference = pcd2np_point_cloud(ds_sub_reference)

print("Number of points in search space (voxels): ", ds_reference.shape[0])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Number of points in search space (voxels):  25


In [4]:
from weighted_fusion import compute_weights, weighted_fusion_filter
from input_output import write_las_file

# Main Code
weights = compute_weights(reference_pc, lst_PC, lst_acc, voxel_size=VOXEL_SIZE)
print("Computing point cloud weights .... Done")

fused_PC_1, point_contrib = weighted_fusion_filter(
    lst_PC, weights, ds_reference, ds_sub_reference, VOXEL_SIZE, k_1, k_2, threshold
)

fused_PC_1 = np.unique(fused_PC_1, axis=0)

file_name = "weighted_fused_filtered_{}_cm_vox.las".format(int(VOXEL_SIZE * 100))
write_las_file(fused_PC_1, os.path.join(output_path, file_name))

Computing point cloud weights .... Done


100%|██████████| 25/25 [00:02<00:00,  9.58it/s]
